In [1]:
import sys
import os

os.chdir('../')
sys.path.append('../')

In [2]:
from cldm.model import create_model

In [3]:
def get_node_name(name, parent_name):
    if len(name) <= len(parent_name):
        return False, ''
    p = name[:len(parent_name)]
    if p != parent_name:
        return False, ''
    return True, name[len(parent_name):]

In [4]:
cldm = create_model(config_path='./models/cldm_v15.yaml')

2023-04-18 05:14:05.969812: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-18 05:14:06.022823: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 05:14:07.172705: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


No module 'xformers'. Proceeding without it.
ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.18.mlp.fc2.bias', 'vision_model.encoder.layers.6.mlp.fc2.bias', 'vision_model.encoder.layers.11.self_attn.q_proj.weight', 'vision_model.encoder.layers.23.self_attn.k_proj.weight', 'vision_model.encoder.layers.22.self_attn.q_proj.weight', 'vision_model.encoder.layers.1.layer_norm1.weight', 'vision_model.encoder.layers.16.layer_norm1.bias', 'vision_model.encoder.layers.9.layer_norm2.bias', 'vision_model.encoder.layers.4.self_attn.q_proj.weight', 'vision_model.encoder.layers.14.self_attn.v_proj.bias', 'text_projection.weight', 'vision_model.encoder.layers.7.self_attn.v_proj.bias', 'vision_model.encoder.layers.13.mlp.fc2.weight', 'vision_model.encoder.layers.22.self_attn.out_proj.bias', 'vision_model.encoder.layers.20.self_attn.k_proj.bias', 'vision_model.encoder.layers.13.layer_norm1.weight', 'vision_model.encoder.layers.23.mlp.fc2.bias', 'visi

Loaded model config from [./models/cldm_v15.yaml]


In [6]:
cldm.__dict__.keys()

dict_keys(['force_null_conditioning', 'num_timesteps_cond', 'scale_by_std', 'training', '_parameters', '_buffers', '_non_persistent_buffers_set', '_backward_hooks', '_is_full_backward_hook', '_forward_hooks', '_forward_pre_hooks', '_state_dict_hooks', '_load_state_dict_pre_hooks', '_load_state_dict_post_hooks', '_modules', 'prepare_data_per_node', 'allow_zero_length_dataloader_with_multiple_devices', '_log_hyperparams', '_dtype', '_device', 'trainer', '_distrib_type', '_device_type', 'use_amp', 'precision', '_example_input_array', '_current_fx_name', '_current_dataloader_idx', '_automatic_optimization', '_truncated_bptt_steps', '_param_requires_grad_state', '_metric_attributes', '_should_prevent_trainer_and_dataloaders_deepcopy', '_loaded_optimizer_states_dict', 'parameterization', 'clip_denoised', 'log_every_t', 'first_stage_key', 'image_size', 'channels', 'use_positional_encodings', 'use_ema', 'use_scheduler', 'v_posterior', 'original_elbo_weight', 'l_simple_weight', 'monitor', 'make

In [7]:
cldm.num_timesteps

1000

In [9]:
scratch_dict = model.state_dict() # 여기에는 학습되지 않은 weights

In [ ]:
for i in scratch_dict.keys():
    print(i)

In [12]:
input_path = '/home/aiteam/tykim/generative_model/framework/diffusion/ControlNet/v1-5-pruned.ckpt'

In [27]:
import torch
pretrained_weights = torch.load(input_path)
if 'state_dict' in pretrained_weights:
    pretrained_weights = pretrained_weights['state_dict']


In [20]:
d2 = list(pretrained_weights['state_dict'].keys())

In [22]:
d1 = list(scratch_dict)

In [23]:
missing_elements = list(set(d1) - set(d2))


In [ ]:
missing_elements
# control_model,... log_var

In [28]:
target_dict = {}
for k in scratch_dict.keys():
    is_control, name = get_node_name(k, 'control_')
    if is_control:
        copy_k = 'model.diffusion_' + name
    else:
        copy_k = k
    if copy_k in pretrained_weights:
        target_dict[k] = pretrained_weights[copy_k].clone()
    else:
        target_dict[k] = scratch_dict[k].clone()
        print(f'These weights are newly added: {k}')

These weights are newly added: logvar
These weights are newly added: control_model.zero_convs.0.0.weight
These weights are newly added: control_model.zero_convs.0.0.bias
These weights are newly added: control_model.zero_convs.1.0.weight
These weights are newly added: control_model.zero_convs.1.0.bias
These weights are newly added: control_model.zero_convs.2.0.weight
These weights are newly added: control_model.zero_convs.2.0.bias
These weights are newly added: control_model.zero_convs.3.0.weight
These weights are newly added: control_model.zero_convs.3.0.bias
These weights are newly added: control_model.zero_convs.4.0.weight
These weights are newly added: control_model.zero_convs.4.0.bias
These weights are newly added: control_model.zero_convs.5.0.weight
These weights are newly added: control_model.zero_convs.5.0.bias
These weights are newly added: control_model.zero_convs.6.0.weight
These weights are newly added: control_model.zero_convs.6.0.bias
These weights are newly added: control

In [4]:
model.load_state_dict(target_dict, strict=True)
output_path  = './test.ckpt'
torch.save(model.state_dict(), output_path)

NameError: name 'model' is not defined

In [9]:
import torch

In [12]:
x_noisy = torch.randn(1, 4, 64, 64)
t = torch.randint(0, cldm.num_timesteps, (1,)).long()

In [33]:
model.summarize()

/tmp/ipykernel_3642461/3704423482.py:1: LightningDeprecationWarning: The `LightningModule.summarize` method is deprecated in v1.5 and will be removed in v1.7. Use `pytorch_lightning.utilities.model_summary.summarize` instead.
  model.summarize()
/home/aiteam/miniconda3/envs/3dfm/lib/python3.8/site-packages/pytorch_lightning/utilities/model_summary.py:464: LightningDeprecationWarning: Argument `mode` in `LightningModule.summarize` is deprecated in v1.4 and will be removed in v1.6. Use `max_depth=1` to replicate `mode=top` behavior.
  rank_zero_deprecation(


  | Name              | Type               | Params
---------------------------------------------------------
0 | model             | DiffusionWrapper   | 859 M 
1 | first_stage_model | AutoencoderKL      | 83.7 M
2 | cond_stage_model  | FrozenCLIPEmbedder | 123 M 
3 | control_model     | ControlNet         | 361 M 
---------------------------------------------------------
1.2 B     Trainable params
206 M     Non-trainable params
1.4 B     Total params
5,710.058 Total estimated model params size (MB)

In [10]:
type(model.model)

ldm.models.diffusion.ddpm.DiffusionWrapper

In [34]:
model.first_stage_model.summarize()

/home/aiteam/miniconda3/envs/3dfm/lib/python3.8/site-packages/pytorch_lightning/utilities/model_summary.py:464: LightningDeprecationWarning: Argument `mode` in `LightningModule.summarize` is deprecated in v1.4 and will be removed in v1.6. Use `max_depth=1` to replicate `mode=top` behavior.
  rank_zero_deprecation(


  | Name            | Type     | Params
---------------------------------------------
0 | encoder         | Encoder  | 34.2 M
1 | decoder         | Decoder  | 49.5 M
2 | loss            | Identity | 0     
3 | quant_conv      | Conv2d   | 72    
4 | post_quant_conv | Conv2d   | 20    
---------------------------------------------
0         Trainable params
83.7 M    Non-trainable params
83.7 M    Total params
334.615   Total estimated model params size (MB)

In [14]:
from ldm.models.ddpm import DDPM


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│ ❱ 1 from ldm.models.ddpm import DDPM                                                             │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'ldm.models.ddpm'

In [ ]:
scr = torch.randn(2, 3, )

In [ ]:
import cv2
import numpy as np
from torch.utils.data import Dataset

class TestDataset(Dataset):
